<a href="https://colab.research.google.com/github/jinsusong/NLP-BERT-Review/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BERT 서브워드 토크나이저 : WordPiece


1. 토큰이 단어 집합에 존재한다.
- 해당 토큰을 분리하지 않는다.

2. 토큰이 단어 집합에 존재하지 않는다.
- 해당 토큰을 서브워드로 분리한다.
- 해당 토큰의 첫번째 서브워드를 제외한 나머지 서브워드들은 앞에 "##"를 붙인 것을 토큰으로 한다.

In [ ]:
#!pip install transformers

In [ ]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # Bert-base의 토크나이저

In [ ]:
#문장을 BERT의 토크나이저가 어떻게 토큰화하는지 봅시다.
result = tokenizer.tokenize('Here is the sentence I want embeddings for.')
print(result)

In [ ]:
# 실제로 BERT의 단어 집합에 특정 단어가 있는지 조회하려면 .vocab[]을 통해서 가능
print(tokenizer.vocab['here'])

In [ ]:
# 해당 단어가 존재하지 않으면 KeyError
print(tokenizer.vocab['embeddings'])

In [ ]:
print(tokenizer.vocab['em'])
print(tokenizer.vocab['##bed'])
print(tokenizer.vocab['##ding'])
print(tokenizer.vocab['##s'])

In [ ]:
# BERT의 단어 집합 전체를 불러와서 살펴보겠습니다. BERT의 단어 집합을 vocabulary.txt에 저장
with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')


In [ ]:
# vocabulary.txt 파일을 직접 열어서 살펴볼 수도 있겠지만, 데이터프레임 형태로 저장해서 확인
df = pd.read_fwf('vocabulary.txt', header=None)
df

In [ ]:
print('단어 집합의 크기 :',len(df))

In [ ]:
df.loc[4667].values[0]

 참고로 BERT에서 사용되는 특별 토큰들과 그와 맵핑되는 정수는 다음과 같습니다.
 - [PAD] - 0
 - [UNK] - 100
 - [CLS] - 101
 - [SEP] - 102
 - [MASK] - 103


In [ ]:
df.loc[102].values[0]

### 포지션 임베딩 (Position Embedding)

- 트랜스포머에서는 포지셔널 인코딩(Positional Encoding)이라는 방법을 통해서 단어의 위치 정보를 표현했습니다. 
- 포지셔널 인코딩은 사인 함수와 코사인 함수를 사용하여 위치에 따라 다른 값을 가지는 행렬을 만들어 이를 단어 벡터들과 더하는 방법

- BERT에서는 이와 유사하지만, 위치 정보를 사인 함수와 코사인 함수로 만드는 것이 아닌 학습을 통해서 얻는 포지션 임베딩(Position Embedding)이라는 방법을 사용


실제 BERT에서는 문장의 최대 길이를 512로 하고 있으므로, 총 512개의 포지션 임베딩 벡터가 학습됩니다. 
결론적으로 현재 설명한 내용을 기준으로는 BERT에서는 총 두 개의 임베딩 층이 사용됩니다. 
단어 집합의 크기가 30,522개인 단어 벡터를 위한 임베딩 층과 문장의 최대 길이가 512이므로 512개의 포지션 벡터를 위한 임베딩 층입니다.

BERT는 세그먼트 임베딩(Segment Embedding)이라는 1개의 임베딩 층을 더 사용

### BERT의 사전 훈련(Pre-training)

 - BERT의 사전 훈련 방법은 크게 두 가지로 나뉩니다. 
 - 첫번째는 마스크드 언어 모델이고, 
 - 두번째는 다음 문장 예측(Next sentence prediction, NSP)

1. 마스크드 언어 모델 ( masked Language Model , MLM)
 - BERT는 사전 훈련을 위해서 인공 신경망의 입력으로 들어가는 입력 텍스트의 15%의 단어를 랜덤으로 마스킹(Masking)합니다. 
 - 인공 신경망에게 이 가려진 단어들을(Masked words) 예측하도록 합니다. 
 - 중간에 단어들에 구멍을 뚫어놓고, 구멍에 들어갈 단어들을 예측하게 하는 식입니다. 
 - 예를 들어 '나는 [MASK]에 가서 그곳에서 빵과 [MASK]를 샀다'를 주고 '슈퍼'와 '우유'를 맞추게 합니다.

2. 다음 문장 예측 (Next Sentence Prediction , NSP)
 - BERT는 두 개의 문장을 준 후에 이 문장이 이어지는 문장인지 아닌지를 맞추는 방식으로 훈련시킵니다. 이를 위해서 50:50 비율로 실제 이어지는 두 개의 문장과 랜덤으로 이어붙인 두 개의 문장을 주고 훈련시킵니

### 세그먼트 임베딩(Segment Embedding)

- BERT는 QA 등과 같은 두 개의 문장 입력이 필요한 태스크를 풀기도 함
- 문장 구분을 위해서 BERT는 세그먼트 임베딩이라는 또 다른 임베딩 층(Embedding layer)을 사용
- 첫번째 문장에는 Sentence 0 임베딩, 두번째 문장에는 Sentence 1 임베딩을 더해주는 방식이며 임베딩 벡터는 두 개만 사용

BERT는 총 3개의 임베딩 층이 사용
- WordPiece Embedding : 실질적인 입력이 되는 워드 임베딩. 임베딩 벡터의 종류는 단어 집합의 크기로 30,522개.
- Position Embedding : 위치 정보를 학습하기 위한 임베딩. 임베딩 벡터의 종류는 문장의 최대 길이인 512개.
- Segment Embedding : 두 개의 문장을 구분하기 위한 임베딩. 임베딩 벡터의 종류는 문장의 최대 개수인 2개.

### BERT를 파인 튜닝(Fine-tuning) 하기

- 사전 학습 된 BERT에 우리가 풀고자 하는 태스크의 데이터를 추가로 학습 시켜서 테스트하는 단계인 파인 튜닝 단계
- 실질적으로 태스크에 BERT를 사용하는 단계에 해당됨

1. 하나의 텍스트에 대한 텍스트 분류 유형(Single Text Classification)
 - BERT를 사용하는 첫번째 유형은 하나의 문서에 대한 텍스트 분류 유형
 - 영화 리뷰 감성 분류, 로이터 뉴스 분류 등과 같이 입력된 문서에 대해서 분류를 하는 유형으로 문서의 시작에 [CLS] 라는 토큰을 입력
 - 사전 훈련 단계에서 다음 문장 예측을 설명할 때, [CLS] 토큰은 BERT가 분류 문제를 풀기위한 특별 토큰, BERT를 실질적으로 사용하는 단계인 파인 튜닝 단계에서도 마찬가지임
 - 텍스트 분류 문제를 풀기 위해서 [CLS] 토큰의 위치의 출력층에서 밀집층(Dense layer) 또는 같은 이름으로는 완전 연결층(fully-connected layer)이라고 불리는 층들을 추가하여 분류에 대한 예측을 하게됨
 


2. 하나의 텍스트에 대한 태깅 작업(Tagging)
 - BERT를 사용하는 두번째 유형은 태깅 작업임. RNN 계열의 신경망들을 이용해서 풀었던 태스크
 - 대표적으로 문장의 각 단어에 품사를 태깅하는 품사 태깅 작업과 개체를 태깅하는 개체명 인식 작업이 있음.
 - 출력층에서는 입력 텍스트의 각 토큰의 위치에 밀집층을 사용하여 분류에 대한 예측을 하게 됨

3. 텍스트의 쌍에 대한 분류 또는 회귀 문제(Text Pair Classification or Regression)
 - BERT는 텍스트의 쌍을 입력으로 받는 태스크도 풀 수 있음. 
 - 텍스트의 쌍을 입력으로 받는 대표적인 태스크로 자연어 추론(Natural language inference)이 있음. 
 - 자연어 추론 문제란, 두 문장이 주어졌을 때, 하나의 문장이 다른 문장과 논리적으로 어떤 관계에 있는지를 분류하는 것
 - 유형으로는 모순 관계(contradiction), 함의 관계(entailment), 중립 관계(neutral)가 있음.
 - 텍스트의 쌍을 입력받는 이러한 태스크의 경우에는 입력 텍스트가 1개가 아니므로, 텍스트 사이에 [SEP] 토큰을 집어넣고, Sentence 0 임베딩과 Sentence 1 임베딩이라는 두 종류의 세그먼트 임베딩을 모두 사용하여 문서를 구분

4. 질의 응답 (Qustion Answering)
 - 텍스트의 쌍을 입력으로 받는 또 다른 태스크로 QA(Question Answering)가 있음. 
 - BERT로 QA를 풀기 위해서 질문과 본문이라는 두 개의 텍스트의 쌍을 입력
 - 태스크의 대표적인 데이터셋으로 SQuAD(Stanford Question Answering Dataset) v1.1이 있음.
 - 데이터셋을 푸는 방법은 질문과 본문을 입력받으면, 본문의 일부분을 추출해서 질문에 답변하는 것 
 - 실제로 이 데이터셋은 영어로 되어있지만 한국어로 예시를 들어보겠습니다. 
 - "강우가 떨어지도록 영향을 주는 것은?" 라는 질문이 주어지고, 
 - "기상학에서 강우는 대기 수증기가 응결되어 중력의 영향을 받고 떨어지는 것을 의미합니다. 강우의 주요 형태는 이슬비, 비, 진눈깨비, 눈, 싸락눈 및 우박이 있습니다." 라는 본문이 주어졌다고 해보겠습니다 
 - 이 경우, 정답은 "중력"이 되어야 합니다.

###  BERT 정보 

- 훈련 데이터는 위키피디아(25억 단어)와 BooksCorpus(8억 단어) ≈ 33억 단어
- WordPiece 토크나이저로 토큰화를 수행 후 15% 비율에 대해서 마스크드 언어 모델 학습
- 두 문장 Sentence A와 B의 합한 길이. 즉, 최대 입력의 길이는 512로 제한
- 100만 step 훈련 ≈ (총 합 33억 단어 코퍼스에 대해 40 에포크 학습)
- 옵티마이저 : 아담(Adam)
- 학습률(learning rate) : 
- 가중치 감소(Weight Decay) : L2 정규화로 0.01 적용
- 드롭 아웃 : 모든 레이어에 대해서 0.1 적용
- 활성화 함수 : relu 함수가 아닌 gelu 함수
- 배치 크기(Batch size) : 256

### 어텐션 마스크(Attention Mask)

- BERT를 실제로 실습하게 되면 어텐션 마스크라는 시퀀스 입력이 추가로 필요합니다. 
- 어텐션 마스크는 BERT가 어텐션 연산을 할 때, 불필요하게 패딩 토큰에 대해서 어텐션을 하지 않도록 실제 단어와 패딩 토큰을 구분할 수 있도록 알려주는 입력입니다. 
- 이 값은 0과 1 두 가지 값을 가지는데, 숫자 1은 해당 토큰은 실제 단어이므로 마스킹을 하지 않는다라는 의미이고, 숫자 0은 해당 토큰은 패딩 토큰이므로 마스킹을 한다는 의미입니다. 
- 실제 단어의 위치에는 1, 패딩 토큰의 위치에는 0의 값을 가지는 시퀀스를 만들어 BERT의 또 다른 입력으로 사용하면 됨